### Closure Applications 

#### Example 1

Let's write a small function that can increment a counter for us - we don't have an incrementor in Python (the ++ operator in Java or C++ for example):

In [ ]:
def counter(initial_value):
    # initial_value is a local variable here
    
    def inc(increment=1):
        nonlocal initial_value
        # initial_value is a nonlocal (captured) variable here
        initial_value += increment
        return initial_value
    
    return inc

In [ ]:
counter1 = counter(0)

In [ ]:
print(counter1(0))

In [ ]:
print(counter1())

In [ ]:
print(counter1())

In [ ]:
print(counter1(8))

In [ ]:
counter2 = counter(1000)

In [ ]:
print(counter2(0))

In [ ]:
print(counter2(1))

In [ ]:
print(counter2())

In [ ]:
print(counter2(220))

As you can see, each closure maintains a reference to the **initial_value** variable that was created when the **counter** function was **called** - each time that function was called, a new local variable **initial_value** was created (with a value assigned from the argument), and it became a nonlocal (captured) variable in the inner scope.

#### Example 2

Let's modify this example to now build something that can run, and maintain a count of how many times we have run some function.

In [ ]:
def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
    
    return inner

In [ ]:
def add(a, b):
    return a + b

In [ ]:
counted_add = counter(add)

And the free variables are:

In [ ]:
counted_add.__code__.co_freevars

We can now call the `counted_add` function:

In [ ]:
counted_add(1, 2)

In [ ]:
counted_add(2, 3)

In [ ]:
def mult(a, b, c):
    return a * b * c

In [ ]:
counted_mult = counter(mult)

In [ ]:
counted_mult(1, 2, 3)

In [ ]:
counted_mult(2, 3, 4)

#### Example 3

Let's take this one step further, and actually store the function name and the number of calls in a global dictionary instead of just printing it out all the time.

In [ ]:
counters = dict()

def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is global
        return fn(*args, **kwargs)
    
    return inner

In [ ]:
counted_add = counter(add)
counted_mult = counter(mult)

Note that `counters` is a **global** variable, and therefore **not** a free variable:

In [ ]:
counted_add.__code__.co_freevars

In [ ]:
counted_mult.__code__.co_freevars

We can now call out functions:

In [ ]:
counted_add(1, 2)

In [ ]:
counted_add(2, 3)

In [ ]:
counted_mult(1, 2, 'a')

In [ ]:
counted_mult(2, 3, 'b')

In [ ]:
counted_mult(1, 1, 'abc')

In [ ]:
print(counters)

Of course this relies on us creating the **counters** global variable first and making sure we are naming it that way, so instead, we're going to pass it as an argument to the **counter** function:

In [ ]:
def counter(fn, counters):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is nonlocal
        return fn(*args, **kwargs)
    
    return inner

In [ ]:
func_counters = dict()
counted_add = counter(add, func_counters)
counted_mult = counter(mult, func_counters)

In [ ]:
counted_add.__code__.co_freevars

As you can see, `counters` is now a free variable.

We can now call our functions:

In [ ]:
for i in range(5):
    counted_add(i, i)

for i in range(10):
    counted_mult(i, i, i)

In [ ]:
print(func_counters)

#### How to stop using global variables?

In [17]:
def mult(a, b, c):
    return a * b * c

def add(a, b):
    return a + b

def counter(fn, counters = dict()):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is nonlocal
        return fn(*args, **kwargs)
    
    return inner

In [18]:
counted_add = counter(add)
counted_mult = counter(mult)

In [19]:
for i in range(5):
    counted_add(i, i)

for i in range(10):
    counted_mult(i, i, i)

#### How to access free variables of a closure?

In [20]:
counted_add.__code__.co_freevars

('cnt', 'counters', 'fn')

In [21]:
counted_mult.__code__.co_freevars

('cnt', 'counters', 'fn')

In [22]:
counted_add.__closure__

(<cell at 0x0000019637D03F70: int object at 0x00007FFCE0731720>,
 <cell at 0x0000019637D03F10: dict object at 0x0000019637D23540>,
 <cell at 0x0000019637D03AF0: function object at 0x00000196387178B0>)

In [23]:
counted_add.__closure__[0].cell_contents

5

In [14]:
for i in range(len(counted_add.__code__.co_freevars)):
    print(f'counted_add: {counted_add.__code__.co_freevars[i]} = {counted_add.__closure__[i].cell_contents}')

counted_add: cnt = 5
counted_add: counters = {'add': 5, 'mult': 10}
counted_add: fn = <function add at 0x0000019636DCCDC0>


In [16]:
for i in range(len(counted_mult.__code__.co_freevars)):
    print(f'counted_add: {counted_mult.__code__.co_freevars[i]} = {counted_mult.__closure__[i].cell_contents}')

counted_add: cnt = 10
counted_add: counters = {'add': 5, 'mult': 10}
counted_add: fn = <function mult at 0x0000019636DCCD30>
